## Create option chain dataframes from source

In [27]:
import pandas as pd 
import os
import re
from tqdm.notebook import tqdm
from IPython.display import display
import numpy as np

import warnings
warnings.filterwarnings("ignore")
import sys
from pathlib import Path
# in jupyter (lab / notebook), based on notebook path
module_path = str(Path.cwd().parents[0] / "src")


if module_path not in sys.path:
    sys.path.append(module_path)
    
from MMAR.MMAR import MMAR

In [6]:
# Get all the entries in raw
dir_list = os.listdir("../data/raw/")

In [10]:
# Select just the needed ones
dir_list = sorted([x for x in dir_list if x.startswith("spy_eod_")])

In [11]:
dir_list

['spy_eod_2018-7fpgd6',
 'spy_eod_2019-eotnx0',
 'spy_eod_2020-kwe0mi',
 'spy_eod_2021-borwkq',
 'spy_eod_2022q1-ww0cra',
 'spy_eod_2022q2-xofvwa',
 'spy_eod_2022q3-we1um1',
 'spy_eod_2022q4-wh2csq']

In [48]:
def clean_df(df:pd.DataFrame, suffix:str|None=None, dest_path:str="../data/processed/chain" )->pd.DataFrame:
    # Fix column names
    new_cols = []
    for col in df.columns:
        new_col = col.strip().replace("[", "").replace("]", "")
        new_cols.append(new_col)
    df.columns = new_cols
    
    # Drop unused columns
    df = df.drop(columns=['QUOTE_UNIXTIME',
    'QUOTE_READTIME',
    'QUOTE_TIME_HOURS',
    'EXPIRE_UNIX',])
    
    # Fix column types
    num_cols = [
        'C_DELTA',
        'C_GAMMA',
        'C_VEGA',
        'C_THETA',
        'C_RHO',
        'C_IV',
        'C_VOLUME',
        'C_LAST',
        'C_BID',
        'C_ASK',
        'P_BID',
        'P_ASK',
        'P_LAST',
        'P_DELTA',
        'P_GAMMA',
        'P_VEGA',
        'P_THETA',
        'P_RHO',
        'P_IV',
        'P_VOLUME',
        ]

    for col in num_cols:
        df[col] = df[col].replace(" ", np.nan)
        df[col] = df[col].astype(float)
        
    # Fix date coulmns
    df.QUOTE_DATE = pd.to_datetime(df.QUOTE_DATE)
    df.EXPIRE_DATE = pd.to_datetime(df.EXPIRE_DATE)
    
    # Sort df
    df = df.sort_values(by=["QUOTE_DATE", "EXPIRE_DATE"]).reindex()
    
    # Save df
    df.to_csv(dest_path+suffix+".csv.gz", index=False, compression="gzip")
        
    return df

In [44]:
def create_chain(path:str,  orig_path:str="../data/raw/")->pd.DataFrame:
    dir_path = orig_path+path+"/"
    file_list = os.listdir(dir_path)
    df = None
    for file in tqdm(file_list):
        print(f"Rading {dir_path+file}")
        tmp_df = pd.read_csv(dir_path+file, low_memory=False)
        if df is None:
            df = tmp_df 
        else:
            df = pd.concat([df, tmp_df])
            
       
    return df

In [49]:

for directory in tqdm(dir_list):
    suffix = re.search(r"(?<=spy_eod_)(.*?)(?=-)", directory).group()
    _ = clean_df(create_chain(directory), suffix)

    
        
    

  0%|          | 0/8 [00:00<?, ?it/s]

  0%|          | 0/12 [00:00<?, ?it/s]

Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201808.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201809.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201802.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201803.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201801.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201810.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201804.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201805.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201811.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201807.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201812.txt
Rading ../data/raw/spy_eod_2018-7fpgd6/spy_eod_201806.txt


  0%|          | 0/12 [00:00<?, ?it/s]

Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201908.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201909.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201901.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201902.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201903.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201907.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201912.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201906.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201910.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201904.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201905.txt
Rading ../data/raw/spy_eod_2019-eotnx0/spy_eod_201911.txt


...

Rading ../data/raw/spy_eod_2022q2-xofvwa/spy_eod_202206.txt
Rading ../data/raw/spy_eod_2022q2-xofvwa/spy_eod_202204.txt
Rading ../data/raw/spy_eod_2022q2-xofvwa/spy_eod_202205.txt


  0%|          | 0/3 [00:00<?, ?it/s]

Rading ../data/raw/spy_eod_2022q3-we1um1/spy_eod_202208.txt
Rading ../data/raw/spy_eod_2022q3-we1um1/spy_eod_202209.txt
Rading ../data/raw/spy_eod_2022q3-we1um1/spy_eod_202207.txt


  0%|          | 0/3 [00:00<?, ?it/s]

Rading ../data/raw/spy_eod_2022q4-wh2csq/spy_eod_202212.txt
Rading ../data/raw/spy_eod_2022q4-wh2csq/spy_eod_202210.txt
Rading ../data/raw/spy_eod_2022q4-wh2csq/spy_eod_202211.txt


### Create option price file

We create a file to be used for ML models

In [6]:
# Get all the entries in processed
dir_list = os.listdir("../data/processed/")

In [7]:
# Select just the needed ones
dir_list = sorted([x for x in dir_list if x.startswith("chain20")])

In [8]:
dir_list

['chain2018.csv.gz',
 'chain2019.csv.gz',
 'chain2020.csv.gz',
 'chain2021.csv.gz',
 'chain2022q1.csv.gz',
 'chain2022q2.csv.gz',
 'chain2022q3.csv.gz',
 'chain2022q4.csv.gz']

In [9]:
def create_opt_cahin_price(filepath)->pd.DataFrame:
    opt_chain = pd.read_csv(filepath, parse_dates=True)
    # Get option prices 
    opt_prices = opt_chain[['QUOTE_DATE', 'UNDERLYING_LAST']].drop_duplicates()
    opt_prices["QUOTE_DATE"] = pd.to_datetime(opt_prices["QUOTE_DATE"])
    opt_prices.columns = ["Date", "Last"]
    opt_prices = opt_prices.set_index("Date")
    
    return opt_prices
    
    

In [10]:
final_df = None
for filename in tqdm(dir_list):
    filepath = "../data/processed/"+filename
    print(f"Reading {filepath}")
    tmp_df = create_opt_cahin_price(filepath)
    if final_df is None:
        final_df = tmp_df 
    else:
        final_df = pd.concat([final_df, tmp_df])
    

  0%|          | 0/8 [00:00<?, ?it/s]

Reading ../data/processed/chain2018.csv.gz
Reading ../data/processed/chain2019.csv.gz
Reading ../data/processed/chain2020.csv.gz
Reading ../data/processed/chain2021.csv.gz
Reading ../data/processed/chain2022q1.csv.gz
Reading ../data/processed/chain2022q2.csv.gz
Reading ../data/processed/chain2022q3.csv.gz
Reading ../data/processed/chain2022q4.csv.gz


In [13]:
final_df = final_df.sort_index()

final_df.to_csv("../data/processed/last_quote.csv")

## Create file for ML models

In [206]:
def generate_diff(num:int, max_iter:int=6)->int:
    """Generator to create alternate sequence

    Example: 
    generate_diff(21) returns 21,20,22,19,23,18,...

    Args:
        num (int): starting point
        max_iter (int): maximum number of iterations 

    Returns:
        int: next element

    Yields:
        Iterator[int]: number
    """
    start = num
    i = 1
    while True:
        yield num
        if i % 2 == 0:
            num =  start + i // 2
        else:
            num = start - (i // 2 + 1)
        i += 1
        if i > max_iter:
            break

In [207]:
def get_ml_df(df:pd.DataFrame, dates_df:pd.DataFrame, diff_date:int=21)->pd.DataFrame:
    # Returning df
    result = df.head(0).copy()

    # Loop for date
    for row in tqdm(dates_df.itertuples(), total=len(dates_df)):
        quote_date, last_quote = row
        quote_date = quote_date.strftime("%Y-%m-%d")
        #Get chain for the date
        query_df = df.query('(QUOTE_DATE==@quote_date)')
        #display(query_df)
        for date_shift in generate_diff(diff_date):
            wk_query = query_df.query('(diff_date==@date_shift)')
            
            if len(wk_query)> 0:
                # Get Strike
                # Index of the minimum difference
                idx = wk_query["diff"].argmin()
                # Append result
                result = pd.concat((result, wk_query.iloc[idx:idx+1]))
                break
    return result

In [208]:
# Get all the entries in processed
dir_list = os.listdir("../data/processed/")

In [209]:
# Select just the needed ones
dir_list = sorted([x for x in dir_list if x.startswith("chain20")])

In [210]:
dir_list

['chain2018.csv.gz',
 'chain2019.csv.gz',
 'chain2020.csv.gz',
 'chain2021.csv.gz',
 'chain2022q1.csv.gz',
 'chain2022q2.csv.gz',
 'chain2022q3.csv.gz',
 'chain2022q4.csv.gz']

In [211]:
ml_df = None
for filename in tqdm(dir_list):
    filepath = "../data/processed/"+filename
    print(f"Reading {filepath}")
    tmp_df = pd.read_csv(filepath, parse_dates=True)
    tmp_df["diff"]=np.abs(tmp_df["UNDERLYING_LAST"]-tmp_df["STRIKE"])
    tmp_df["diff_date"] = (pd.to_datetime(tmp_df["EXPIRE_DATE"])-pd.to_datetime(tmp_df["QUOTE_DATE"])).dt.days
    tmp_df = tmp_df[[
            "QUOTE_DATE",
            "EXPIRE_DATE",
            "UNDERLYING_LAST",
            "STRIKE",
            "C_LAST",
            "C_BID",
            "C_ASK",
            "C_IV",
            "C_VOLUME",
            "diff",
            "diff_date"
        ]]
    ret_df = get_ml_df(tmp_df, final_df)
    
    if ml_df is None:
        ml_df = ret_df 
    else:
        ml_df = pd.concat([ml_df, ret_df])
    

  0%|          | 0/8 [00:00<?, ?it/s]

Reading ../data/processed/chain2018.csv.gz


  0%|          | 0/1258 [00:00<?, ?it/s]

Reading ../data/processed/chain2019.csv.gz


  0%|          | 0/1258 [00:00<?, ?it/s]

...

  0%|          | 0/1258 [00:00<?, ?it/s]

Reading ../data/processed/chain2022q3.csv.gz


  0%|          | 0/1258 [00:00<?, ?it/s]

Reading ../data/processed/chain2022q4.csv.gz


  0%|          | 0/1258 [00:00<?, ?it/s]

In [212]:
ml_df

,QUOTE_DATE,EXPIRE_DATE,UNDERLYING_LAST,STRIKE,C_LAST,C_BID,C_ASK,C_IV,C_VOLUME,diff,diff_date
423,2018-01-02,2018-01-24,268.81,269.0,1.56,1.72,1.76,0.07134,80.0,0.19,22
2680,2018-01-03,2018-01-24,270.47,270.0,2.15,2.10,2.15,0.07361,271.0,0.47,21
4917,2018-01-04,2018-01-24,271.56,272.0,1.67,1.57,1.61,0.06871,392.0,0.44,20
7390,2018-01-05,2018-01-26,273.41,273.0,2.21,2.23,2.28,0.07675,NaN,0.41,21
9795,2018-01-08,2018-01-31,273.94,274.0,2.08,2.07,2.09,0.07593,490.0,0.06,23
...,...,...,...,...,...,...,...,...,...,...,...
244603,2022-12-23,2023-01-13,382.88,383.0,8.00,7.97,7.99,0.20485,594.0,0.12,21
248071,2022-12-26,2023-01-13,382.91,383.0,7.96,7.91,8.00,0.22174,603.0,0.09,18
255230,2022-12-28,2023-01-20,376.71,377.0,8.75,8.73,8.76,0.21881,807.0,0.29,23
258680,2022-12-29,2023-01-20,383.33,383.0,8.50,8.47,8.50,0.20722,3413.0,0.33,22


In [215]:
# Set date index
ml_df = ml_df.set_index(pd.to_datetime(ml_df["QUOTE_DATE"]))

In [217]:
# Rename index
ml_df.index.name = "Date"

In [221]:
# Sort df
ml_df = ml_df.sort_index()

In [222]:
# Save to file
ml_df.to_csv("../data/processed/ml_df.csv")

## Generate expected price using Monte Carlo simulation

In [223]:
spy = pd.read_csv("../data/raw/spy.csv", parse_dates=True, index_col=0)

Get interest rate (form Fred St Louis: https://fred.stlouisfed.org/series/DGS1MO)

In [229]:
interest_rate = pd.read_csv("../data/raw/DGS1MO.csv", parse_dates=True, index_col=0, na_values=["."])

In [248]:
def get_call_price(prices, strike, r, T):
    values = prices[prices > strike]
    prob = len(values) / len(prices)
    exp_val = np.mean(values)
    #print(f"Probability expected price {exp_val:.2f} > {strike:.2f} is {prob:.2%}")
    return np.exp(-r * T) * prob * (exp_val - strike), prob

In [249]:
exp_price = []
exp_call_prices = []
date_idx = []
rates = []
strikes = []
last_quotes = []
days = []
probs = []
num_sim = 250_000
prev_ir = 0
for row in tqdm(ml_df.itertuples(), total=len(ml_df)):
    # Get elements
    final_date = row.Index
    S0 = row.UNDERLYING_LAST
    num_steps = row.diff_date
    strike = row.STRIKE
    r = interest_rate.loc[final_date,"DGS1MO"]
    if r is None:
        r = prev_ir
    else:
        prev_ir = r
    # Filter df
    wk_df = spy.loc[:final_date,]
    # Generate simulation
    mmar = MMAR(wk_df["Close"], volume=wk_df["Volume"])
    simulation = mmar.get_MMAR_MC(S0, n=num_steps, num_sim=num_sim)
    exp_price.append(simulation[:, -1:].mean())
    exp_call_price, prob = get_call_price(simulation[:, -1:], strike, r / 100, num_steps / 252)
    exp_call_prices.append(exp_call_price)
    strikes.append(strike)
    last_quotes.append(S0)
    days.append(num_steps)
    probs.append(prob)
    rates.append(r)
    date_idx.append(final_date)
    


  0%|          | 0/1253 [00:00<?, ?it/s]

The series has been adjusted: the orginal size was 3273, the new size is 3272 with 6 dividers.
The series has been adjusted: the orginal size was 3274, the new size is 3272 with 6 dividers.
The series has been adjusted: the orginal size was 3275, the new size is 3272 with 6 dividers.
The series has been adjusted: the orginal size was 3277, the new size is 3276 with 34 dividers.
The series has been adjusted: the orginal size was 3279, the new size is 3278 with 6 dividers.
The series has been adjusted: the orginal size was 3281, the new size is 3280 with 18 dividers.
The series has been adjusted: the orginal size was 3283, the new size is 3282 with 6 dividers.
The series has been adjusted: the orginal size was 3284, the new size is 3282 with 6 dividers.
The series has been adjusted: the orginal size was 3287, the new size is 3286 with 6 dividers.
The series has been adjusted: the orginal size was 3291, the new size is 3290 with 14 dividers.
The series has been adjusted: the orginal size 

In [250]:
exp_price_df = pd.DataFrame({"last_quote":last_quotes,
                             "strike":strikes,"exp_price":exp_price, 
                             "exp_call_price":exp_call_prices, 
                             "days":days, 
                             "r":rates,
                             "prob_itm":probs}, 
                            index = pd.Index(name="Date", data=date_idx))

In [251]:
exp_price_df

,last_quote,strike,exp_price,exp_call_price,days,r,prob_itm
Date,,,,,,,
2018-01-02,268.81,269.0,269.056555,4.793532,22,1.29,0.492236
2018-01-03,270.47,270.0,270.699589,4.883035,21,1.29,0.515376
2018-01-04,271.56,272.0,271.766754,4.313546,20,1.28,0.483992
2018-01-05,273.41,273.0,273.642665,4.894046,21,1.27,0.514020
2018-01-08,273.94,274.0,274.225205,4.864488,23,1.30,0.498300
...,...,...,...,...,...,...,...
2022-12-23,382.88,383.0,383.411772,8.436301,21,3.80,0.497404
2022-12-26,382.91,383.0,383.457438,NaN,18,NaN,0.498708
2022-12-28,376.71,377.0,377.291314,8.525205,23,3.86,0.494764


In [252]:
# Save file
exp_price_df.to_csv("../data/processed/exp_price.csv")
